In [ ]:
from utils.get_only_information_systems_students import get_only_information_systems_students
from utils.treat_dataset import treat_dataset
import pandas as pd

dataset_path = "datasets/global.csv"

df = pd.read_csv(dataset_path)
df = df.sort_values(by=["student_id", "period", "discipline"])

# Remoção do "traço" e do crédito da disciplina no nome da mesma
df["discipline"] = df["discipline"].str.split(" - ").str[0].str.strip()
df["discipline"] = df["discipline"].str.split(" -72").str[0].str.strip() # Remoção para caso especial em "Gestão da Qualidade de Software"

print(f"Quantidade de registros: {len(df)}\n")
print(df.head())

Quantidade de registros: 35995

       id  student_id                      birthdate sex                city  \
2138    1       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
5665    3       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
9160    2       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
28710   4       99955  1989-01-06 04:00:00.000 -0200   M  Faxinal do Soturno   
28312   8       99955  1989-01-06 04:00:00.000 -0200   M  Faxinal do Soturno   

                   course  period week_day                       discipline  \
2138   Ciências Contábeis  2021/1   Quarta              Matemática Aplicada   
5665   Ciências Contábeis  2021/2  Segunda      Contabilidade Intermediária   
9160   Ciências Contábeis  2022/1    Terça           Contabilidade Avançada   
28710  Ciências Contábeis  2025/1  Segunda       Contabilidade Introdutória   
28312  Ciências Contábeis  2025/1   Quarta  Legislação e Ética Profissional   

         sta

In [74]:
# Remoção de colunas não utilizadas
df = treat_dataset(df)

# Filtragem de registros apenas de alunos de Sistemas de Informação
df = get_only_information_systems_students(df)

print(f"Quantidade de registros: {len(df)}\n")
print(df.head())

Quantidade de registros: 4604

       id  student_id                  course   period  \
133   248      100004  Sistemas de Informação   2020/2   
111   249      100004  Sistemas de Informação   2020/2   
134   247      100004  Sistemas de Informação   2020/2   
653   246      100004  Sistemas de Informação   2020/2   
1918  250      100004  Sistemas de Informação  2020/F2   

                                     discipline                    status  \
133             Gestão da Qualidade de Software                  Aprovado   
111                 Laboratório de Algoritmo II  Reprovado por Frequência   
134   Organização e Arquitetura de Computadores                  Aprovado   
653                         Psicologia do Líder                  Aprovado   
1918       Programação para Dispositivos Móveis                  Aprovado   

        g1    g2  final_grade  class_skips  
133    9.7   8.0          8.8            0  
111    0.0   0.0          0.0           24  
134    8.3   8.2      

In [75]:
# Salva dataset atual filtrado apenas para Sistemas de Informação
df.to_csv("datasets/sistemas_de_informacao.csv", index=False, encoding="utf-8")

In [76]:
# Mostra a quantidade de disciplinas (com agrupamento)
discipline_counts = df["discipline"].value_counts()

print(f"Quantidade de disciplinas: {len(discipline_counts)}\n")
print("Disciplina X Quantidade\n")
print(discipline_counts)

Quantidade de disciplinas: 110

Disciplina X Quantidade

discipline
Estrutura de Dados                                                                                 201
Organização e Arquitetura de Computadores                                                          193
Competência Competitiva no Universo da Tecnologia                                                  178
Probabilidade e Estatística                                                                        165
Laboratório de Algoritmos II                                                                       151
                                                                                                  ... 
Trabalho de Conclusão de Curso em Administração                                                      1
Cultura Humanista e Igualdade nas Relações Étnico-Raciais e História Afro-Brasileira e Indígena      1
O Mundo do Trabalho e a Administração                                                                1
Perso

In [77]:
# Salva dataset atual apenas registros de disciplinas agrupadas para fins de análise
discipline_counts_df = discipline_counts.reset_index()
discipline_counts_df.columns = ["discipline", "count"]
discipline_counts_df.to_csv("datasets/sistemas_de_informacao_disciplines_grouped.csv", index=False, encoding="utf-8")

In [ ]:
# Cadeiras necessárias para a previsão de aprovação (até o 3° semestre)
required_disciplines = [
    "Fundamentos de Sistemas de Informação",
    "Laboratório de Algoritmos",
    "Laboratório de Algoritmos I",
    "Pensamento Computacional",
    "Estrutura de Dados",
    "Organização e Arquitetura de Computadores",
    "Sistemas de Informação Gerenciais",
    "Gerenciamento de Serviços de TI",
    "Classificação e Pesquisa de Dados",
    "Fundamentos de Banco de Dados",
    "Sistemas Operacionais",
    # Matemática
    "Cálculo Numérico",
    "Probabilidade e Estatística",
    "Matemática Discreta",
    # FOIL
    "O Mundo do Trabalho e a Tecnologia",
    "Competência Competitiva no Universo da Tecnologia",
    "Personalidade e Carreiras Tecnológicas",
    # INGLÊS
    "Língua Inglesa I",
    "Língua Inglesa II",
    "Língua Inglesa III",
]

# Agrupamento por aluno e disciplinas
student_disciplines = df.groupby("student_id")["discipline"].apply(set)

# Filtragem do dataset para manter registros de alunos que tenham cursado ao menos uma das disciplinas listadas
valid_students = student_disciplines[student_disciplines.apply(lambda d: len(set(required_disciplines).intersection(d)) > 0)].index

# Filtragem do dataset original mantendo todos os registros desses alunos
filtered_df = df[df["student_id"].isin(valid_students)].copy()

# Nova filtragem mantendo apenas cadeiras aprovadas
filtered_df = filtered_df[filtered_df["status"] == "Aprovado"].copy()

print(f"Registros viáveis para previsão de aprovação: {len(filtered_df)}\n")
print(filtered_df.head())

Registros viáveis para previsão de aprovação: 2591

       id  student_id                  course   period  \
133   248      100004  Sistemas de Informação   2020/2   
134   247      100004  Sistemas de Informação   2020/2   
653   246      100004  Sistemas de Informação   2020/2   
1918  250      100004  Sistemas de Informação  2020/F2   
2274  251      100004  Sistemas de Informação   2021/1   

                                     discipline    status    g1    g2  \
133             Gestão da Qualidade de Software  Aprovado   9.7   8.0   
134   Organização e Arquitetura de Computadores  Aprovado   8.3   8.2   
653                         Psicologia do Líder  Aprovado   6.5   9.0   
1918       Programação para Dispositivos Móveis  Aprovado  10.0  10.0   
2274     Análise e Projeto Orientados a Objetos  Aprovado   8.0   8.5   

      final_grade  class_skips  
133           8.8            0  
134           8.2            4  
653           7.7            0  
1918         10.0           